<a href="https://colab.research.google.com/github/esheshka/team_one_project/blob/main/%D0%A7%D1%82%D0%BE_%D1%82%D0%BE_%D0%B7%D0%B0%D1%87%D0%B5%D0%BC_%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>